In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split

2025-03-05 17:46:43.434746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741186003.454801    1407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741186003.460927    1407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:46:43.481070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DatasetVer2 = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/HighCorrelationDataset-V-2-1.csv")

In [3]:
TrainingData = DatasetVer2.loc[:, ~DatasetVer2.columns.isin(['Category','Label'])]

LabelData = DatasetVer2.loc[:,DatasetVer2.columns.isin(['Label'])]
LabelData = LabelData.squeeze()

In [8]:
Dataset = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/LatestDataset-V-2.csv")

CategoryTrainingData = Dataset.loc[:, ~Dataset.columns.isin(['Category','Label'])]

CategoryLabelData = Dataset.loc[:,Dataset.columns.isin(['Category'])]
CategoryLabelData = CategoryLabelData.squeeze()

In [3]:
ClassificationDataTraining, ClassificationDataTesting, ClassificationLabelTraining, ClassificationLabelTesting = train_test_split(CategoryTrainingData, CategoryLabelData, random_state=138, train_size=0.8, stratify=CategoryLabelData)

In [12]:
ClassificationDataTraining, ClassificationDataTesting, ClassificationLabelTraining, ClassificationLabelTesting = train_test_split(
    CategoryTrainingData, CategoryLabelData, random_state=138, train_size=0.8, stratify=CategoryLabelData
)

# Define directory path
directory = "/mnt/e/Projects/Anomaly Detection/Dataset/Model/"

# Convert to DataFrames if not already
ClassificationDataTraining_df = pd.DataFrame(ClassificationDataTraining)
ClassificationLabelTraining_df = pd.DataFrame(ClassificationLabelTraining, columns=['Label'])
ClassificationDataTesting_df = pd.DataFrame(ClassificationDataTesting)
ClassificationLabelTesting_df = pd.DataFrame(ClassificationLabelTesting, columns=['Label'])

# Save to CSV with original names in the specified directory
ClassificationDataTraining_df.to_csv(directory + 'ClassificationDataTraining.csv', index=False)
ClassificationLabelTraining_df.to_csv(directory + 'ClassificationLabelTraining.csv', index=False)
ClassificationDataTesting_df.to_csv(directory + 'ClassificationDataTesting.csv', index=False)
ClassificationLabelTesting_df.to_csv(directory + 'ClassificationLabelTesting.csv', index=False)

In [4]:
DetectionDataTraining, DetectionDataTesting, DetectionLabelTraining, DetectionLabelTesting = train_test_split(TrainingData, LabelData, random_state=138, train_size=0.8, stratify=LabelData)

In [13]:
ModelVer3 = Sequential(
    [
    tf.keras.layers.Input((31, )),
    tf.keras.layers.Dense(units=128,activation='relu', name='First'),
    tf.keras.layers.Dense(units=64,activation='relu', name='Second'),
    tf.keras.layers.Dense(units=32,activation='relu', name='Third'),
    tf.keras.layers.Dense(units=16,activation='relu', name='Fourth'),
    tf.keras.layers.Dense(units=8,activation='relu', name='Fifth'),
    tf.keras.layers.Dense(units=4,activation='relu', name='Sixth'),
    tf.keras.layers.Dense(units=2,activation='softmax', name='Output')

    ], name = "DetectionANN"
)

ModelVer3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
    )

ModelVer3.summary()

Model: "DetectionANN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ First (Dense)                   │ (None, 128)            │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Second (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Third (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fourth (Dense)                  │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fifth (Dense)                   │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sixth (Dense)                   │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 2)              │            10 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,142 (59.15 KB)

 Trainable params: 15,142 (59.15 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
DetectionDataTraining = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/Model/DataTraining.csv")
DataTesting = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/Model/DataTesting.csv")
DetectionLabelTraining = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/Model/LabelTraining.csv")
LabelTesting = pd.read_csv(r"/mnt/e/Projects/Anomaly Detection/Dataset/Model/LabelTesting.csv")

In [5]:
ModelVer3 = tf.keras.models.load_model(r"/mnt/e/Projects/Anomaly Detection/Models/Detection3.keras")

I0000 00:00:1741186036.476710    1407 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4072 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [6]:
ModelVer3.fit(
    DetectionDataTraining, DetectionLabelTraining, 
    epochs=200,
    batch_size=4096,
)

Epoch 1/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0255
Epoch 2/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - loss: 0.0256
Epoch 3/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.0264
Epoch 4/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 0.0262
Epoch 5/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 0.0266
Epoch 6/200
473/473 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.0262
Epoch 7/200
366/473 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0261

KeyboardInterrupt: 

In [ ]:
ModelVer3.save(r"/mnt/e/Projects/Anomaly Detection/Models/Detection4.keras")

In [4]:
ModelVer5 = Sequential(
    [
    tf.keras.layers.Input((29, )),
    tf.keras.layers.Dense(units=128,activation='relu', name='First'),
    tf.keras.layers.Dense(units=64,activation='relu', name='Second'),
    tf.keras.layers.Dense(units=32,activation='relu', name='Third'),
    tf.keras.layers.Dense(units=16,activation='relu', name='Fourth'),
    tf.keras.layers.Dense(units=8,activation='relu', name='Fifth'),
    tf.keras.layers.Dense(units=4,activation='relu', name='Sixth'),
    tf.keras.layers.Dense(units=10,activation='softmax', name='Output')

    ], name = "Class"
)

ModelVer5.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
    )

ModelVer5.summary()

I0000 00:00:1741124566.720906   22040 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4080 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "Class"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ First (Dense)                   │ (None, 128)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Second (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Third (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fourth (Dense)                  │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fifth (Dense)                   │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sixth (Dense)                   │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 10)             │            50 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,926 (58.30 KB)

 Trainable params: 14,926 (58.30 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
ModelVer5.fit(
    ClassificationDataTraining, ClassificationLabelTraining, 
    epochs=200,
    batch_size=2048,
)

Epoch 1/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0189
Epoch 2/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0194
Epoch 3/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0187
Epoch 4/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0187
Epoch 5/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0193
Epoch 6/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0188
Epoch 7/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0190
Epoch 8/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0191
Epoch 9/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0175
Epoch 10/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0187
Epoch 11/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0194
Epoch 12/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0184
Epoch 13/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0188
Epoch 14/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0187
Epoch 15/200
946/946 ━━━━━━━━━━━━━━━━━━━━ 6

In [7]:
ModelVer5 = tf.keras.models.load_model(r"/mnt/e/Projects/Anomaly Detection/Models/Classification3.keras")

I0000 00:00:1741123494.534446   16785 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4080 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [ ]:
ModelVer5.fit(
    ClassificationDataTraining, ClassificationLabelTraining, 
    epochs=100,
)

In [8]:
ModelVer5.save(r"/mnt/e/Projects/Anomaly Detection/Models/Classification4.keras")

In [ ]:
ClassificationDataTesting.to_csv(r"E:\Projects\Anomaly Detection\Dataset\Model\ClassificationDataTesting.csv", index=False)
ClassificationLabelTesting.to_csv(r"E:\Projects\Anomaly Detection\Dataset\Model\ClassificationLabelTesting.csv", index=False)

: 